# 大課題 毒キノコの判定 SVM

## 導入
### 【課題】SVMとは何か
SVMとは何か説明できるようになることがテキストの目標のひとつでした。
以下の要素を含みながらSVMについて簡潔に説明してください。
- どのようなときに使うのか
- どのような仕組みなのか
- どういった利点欠点があるのか

### 答え：
#### どのようなときに使うのか
　一般的なデータから複雑なデータまで、線形、非線形の分類や回帰を行いたい場合に使う

#### どのような仕組みなのか
 　データの中で、互いに異なる性質を持つサンプル2点を境界で区切る。このとき、最も近い距離にあるサンプル2点をサポートベクタ、サポートベクタ間の距離をマージンと呼ぶ。境界は、線や平面であり、高次元の場合には超平面になる。  
　分類の場合には、マージンをどれだけ広くとれるか、データをどれだけ誤りなく区切ることができるか、の主に２つの基準からサポートベクタと境界を求める。  
　回帰の場合には、マージンをどれだけ狭くとれるか、データをどれだけマージン内に収めることができるか、の主に２つの基準からサポートベクタと境界を求める
 
#### どういった利点欠点があるのか
##### 利点
- マージンの最大化(分類)・最小化(回帰)が考慮されているので、未学習データに対しての識別性能が高い。つまり汎化能力が高い
- 高次元空間への写像を行うことで複雑な問題に対処できるので、特徴量が多くても精度が高い  

##### 欠点
- 多クラスの分類には一工夫が必要になる
- 学習サンプル数が増えると計算量が増大する

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display

from sklearn import svm

from sklearn import preprocessing as sp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, recall_score, precision_score

# 表示を省略しない
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

## データの取得
UCIのMushroom Data Setを使用します。  
[UCI Machine Learning Repository: Mushroom Data Set](https://archive.ics.uci.edu/ml/datasets/Mushroom)  

23種類のキノコについての8124のサンプルが集められたデータセットです。特徴量は22個で、ターゲットはedible（食べられる）、poisonous（毒がある）の二種類です。
### 【課題】データを取得する
pandasで上記ページにあるファイルを読み込んでください。

In [2]:
# データを読み込むコードを記述
# 'target'を除くカラム名は、データに付属するData Set Descriptionから取り出した
data = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data', 
    names=['target', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 
           'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 
          'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 
          'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 
          'spore-print-color', 'population', 'habitat'], 
    encoding='utf-8')

In [3]:
data.head()

,target,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


## 前処理
### 【課題】データの変換
このデータセットではターゲットも特徴量も全て文字列なので、SVMで利用できるようにそれらを整数に変換しましょう。sklearn.preprocessingの中のメソッドが利用できます。

In [4]:
#文字列から整数へ変換するコードを記述
le = sp.LabelEncoder()
for column in data.columns:
    data[column] = le.fit_transform(data[column])

### 【課題】データセットの分割
testデータとtrainデータに分割しましょう。

In [5]:
#データセットを分割するコードの記述
X = data.drop('target', axis=1)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### 【課題】標準化
SVMを行う前にデータの標準化をしましょう。Feature Scalingのテキストを参考にしてください。また、なぜSVMを行う際に標準化が必要なのか説明してください。

In [6]:
#標準化を行うコードを記述
stdsc = sp.StandardScaler()

X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

#### なぜSVMを行う際に標準化が必要なのか
　SVMは、勾配法を用いたモデルであり、モデルの各係数の大小は、対応する特徴量の値の大小に影響される。一般的に、特徴量ごとの測定単位や尺度はバラバラであるため、そのままモデルをつくった場合の係数が偏り、モデルの信頼性が下がる。このため、SVMのモデルをつくる際には、全ての特徴量を統一の基準で扱わなければならない。これ故に、特徴量の平均を0、分散を1の統一基準で扱える、標準化が必要である

## ハイパーパラメータの調整
### 【課題】カーネルとは何か
SVMにおいて重要になってくるカーネルとは何でしょうか。簡潔に説明してください。また、カーネルはどう使い分けるのが良いか調べて記述してください。

### 答え：
#### カーネルとは何か
実データを高次元空間へと写像する際の内積計算を肩代わりする関数のこと
 
#### どう使い分けるのが良いか
　計算コストのかかる内積計算を肩代わりするという性質上、そのまま内積計算するより以下の計算コストで写像を実現できるカーネルが望ましい。  
　しかし、一般的に用いられるRBFカーネルは高い表現力ゆえに比較的過学習しやすく、多項式カーネルは表現力は高いがハイパーパラメータが4つでグリッドサーチの負担が大きい、 線形カーネルはそのまま内積計算を行う故に高次元化は確実であるが計算コストがかかる、といったように各カーネルには一長一短ある。  
　各カーネルの特性を理解の上で、計算コスト、表現力、過学習を防止できるか、グリッドサーチでハイパーパラメータを特定する処理コスト、などを考慮して使い分けるのが良いと考える

### 【課題】コストペナルティCとは何か
ハイパーパラメータとして設定するコストペナルティCについて簡潔に説明してください。

### 答え：
　SVMの誤分類に対するペナルティ。これを大きく設定するほど、誤分類に対して厳しいモデルになる。誤分類に厳しければマージンを広く取れなくなるため、過学習になり易いモデルになるとも云える

### 【課題】ハイパーパラメータを調整する
この大課題の中で一番重要な部分です。  
テキストでは実装しませんでしたが、SVMには様々なハイパーパラメータが存在し、その設定によって結果に違いが出ることを学びました。グリッドサーチを行うことでハイパーパラメータを決定してください。なお、実行にはある程度時間がかかることが予想されます。  
カーネルもハイパーパラメータのひとつと考えられます。

In [7]:
#グリッドサーチのコードを記述
svc = svm.SVC()

# 各パラメータは、全カーネルを含めて十数回調整しました
# そのうちに1.0の結果が出たので、以下に落ち着けました
parameters = {
    'C': [5.0, 5.1, 5.2], 
    'gamma': [0.25/22, 0.30/22, 0.35/22]}

clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(X_train_std, y_train)

print(clf.best_params_)
print(clf.best_score_)

{'C': 5.0, 'gamma': 0.011363636363636364}
1.0


## SVMの実行
### 【課題】学習およびテスト
選択したハイパーパラメータを利用して、学習およびテストを行ってください。精度も表示しましょう。

In [8]:
#SVMの学習およびテストをするコードを記述

# 学習
svc_h = svm.SVC(C=5.0, gamma=0.30/22)
svc_h.fit(X_train_std, y_train)

# 予測
y_pred = svc_h.predict(X_test_std)

# テスト（accuracy, recall, precision）
# accuracy
print('(予測[毒]結果[毒] + 予測[食]結果[食]) / 全体： ' + str(accuracy_score(y_test, y_pred)))

# recall
print('予測[毒]結果[毒] / 結果[毒]： ' + str(recall_score(y_test, y_pred)))

# precision
print('予測[毒]結果[毒] / 予測[毒]： ' + str(precision_score(y_test, y_pred)))

(予測[毒]結果[毒] + 予測[食]結果[食]) / 全体： 1.0
予測[毒]結果[毒] / 結果[毒]： 1.0
予測[毒]結果[毒] / 予測[毒]： 1.0
